In [ ]:
import os
import pandas as pd
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta
from massive import RESTClient

api_key = os.getenv("MASSIVE_API_KEY")
client = RESTClient(api_key)
TICKER = "SPY"
ADJUSTED = True

def get_last_n_trading_days(symbol: str,
                            n_days: int = 30,
                            lookback_days: int = 90) -> pd.DataFrame:
    """
    Fetch the last n_days trading bars for the given symbol.

    We request a wider calendar window (lookback_days) and then
    keep the most recent n_days trading bars.
    """
    end = date.today()
    start = end - timedelta(days=lookback_days)

    bars = list(
        client.list_aggs(
            ticker=symbol,
            multiplier=1,
            timespan="day",
            from_=start.isoformat(),
            to=end.isoformat(),
            adjusted=ADJUSTED,
            sort="asc",
            limit=50000,
        )
    )

    if not bars:
        raise RuntimeError(
            f"No bars returned for {symbol} between {start} and {end}"
        )

    rows = []
    for b in bars:
        # Massive returns timestamp in ms; convert to UTC date
        trade_dt = datetime.utcfromtimestamp(b.timestamp / 1000).date()
        rows.append(
            {
                "date": trade_dt.isoformat(),
                "open": b.open,
                "high": b.high,
                "low": b.low,
                "close": b.close,
                "volume": b.volume,
            }
        )

    df = pd.DataFrame(rows).sort_values("date")

    # Keep only the last n_days trading sessions
    if len(df) > n_days:
        df = df.tail(n_days)

    df = df.reset_index(drop=True)
    return df


if __name__ == "__main__":
    df = get_last_n_trading_days(TICKER, n_days=30)

    # Save next to current working directory
    out_file = os.path.join(os.getcwd(), "benchmark_history.csv")
    df.to_csv(out_file, index=False)

    print(f"Saved last 30 trading days for {TICKER} to {out_file}")
    print(df)


Fetching last 30 trading days for 1505 symbols (lookback 90 calendar days, ref=2025-11-16)
ARE: 30 rows
ALGN: 30 rows
AOS: 30 rows
ACN: 30 rows
ALB: 30 rows
ALLE: 30 rows
AMD: 30 rows
LNT: 30 rows
ABNB: 30 rows
GOOGL: 30 rows
ADBE: 30 rows
ABBV: 30 rows
AFL: 30 rows
ABT: 30 rows
APD: 30 rows
AKAM: 30 rows
MMM: 30 rows
AES: 30 rows
ALL: 30 rows
A: 30 rows
AIG: 30 rows
MO: 30 rows
AMCR: 30 rows
AEE: 30 rows
AMT: 30 rows
AWK: 30 rows
AEP: 30 rows
GOOG: 30 rows
AMGN: 30 rows
AXP: 30 rows
AMZN: 30 rows
ADI: 30 rows
APA: 30 rows
APO: 30 rows
APH: 30 rows
AON: 30 rows
AAPL: 30 rows
AME: 30 rows
AMAT: 30 rows
AMP: 30 rows
ACGL: 30 rows
APP: 30 rows
APTV: 30 rows
ATO: 30 rows
ANET: 30 rows
ADM: 30 rows
AVY: 30 rows
ADSK: 30 rows
AIZ: 30 rows
AVB: 30 rows
T: 30 rows
AZO: 30 rows
BALL: 30 rows
ADP: 30 rows
BDX: 30 rows
AXON: 30 rows
BKR: 30 rows
BAC: 30 rows
BAX: 30 rows
AJG: 30 rows
BRK-B: 0 rows
BBY: 30 rows
TECH: 30 rows
BLK: 30 rows
BX: 30 rows
BKNG: 30 rows
BIIB: 30 rows
BSX: 30 rows
XYZ: 30

,symbol,date,open,high,low,close,volume,vwap
570,A,2025-10-06,141.51,141.935,139.90,141.61,1538402.0,141.3568
571,A,2025-10-07,141.86,142.270,138.44,138.56,1444593.0,139.3369
572,A,2025-10-08,140.36,142.440,138.97,140.81,1698819.0,141.3061
573,A,2025-10-09,140.28,140.940,138.48,140.11,1149998.0,139.6371
574,A,2025-10-10,139.66,140.575,136.29,136.62,1968156.0,137.3252
